# 创建自定义 Estimator
本文档介绍了自定义 Estimator。具体而言，本文档介绍了如何创建自定义 Estimator 来模拟预创建的 Estimator DNNClassifier 在解决鸢尾花问题时的行为。

## 预创建的 Estimator 与自定义 Estimator
预创建的 Estimator 是 tf.estimator.Estimator 基类的子类，而自定义 Estimator 是 tf.estimator.Estimator 的实例

预创建的 Estimator 已完全成形。不过有时，您需要更好地控制 Estimator 的行为。这时，自定义 Estimator 就派上用场了。您可以创建自定义 Estimator 来完成几乎任何操作。如果您需要以某种不寻常的方式连接隐藏层，则可以编写自定义 Estimator。如果您需要为模型计算独特的指标，也可以编写自定义 Estimator。基本而言，如果您需要一个针对具体问题进行了优化的 Estimator，就可以编写自定义 Estimator。

In [1]:
import tensorflow as tf
import os
import pandas as pd
import iris_data

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL =  "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES =  ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
def load_data(label_name = 'Species'):
    train_path = tf.keras.utils.get_file(fname=os.path.basename(TRAIN_URL),
                                        origin = TRAIN_URL)
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    
    train_features,train_label = train,train.pop(label_name)
    
    test_path = tf.keras.utils.get_file(fname=os.path.basename(TEST_URL),
                                       origin=TEST_URL)
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    
    test_features,test_label = test,test.pop(label_name)
    
    return (train_features,train_label),(test_features,test_label)

In [3]:
(train_x,train_y),(test_x,test_y) = load_data()
print(train_x[:5])

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


## 编写输入函数


In [4]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.shuffle(1000).repeat(None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

## 创建特征列

In [5]:
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key))

## 编写模型函数
前两个参数是从输入函数中返回的特征和标签批次；也就是说，features 和 labels 是模型将使用的数据的句柄。mode 参数表示调用程序是请求训练、预测还是评估。

调用程序可以将 params 传递给 Estimator 的构造函数。传递给构造函数的所有 params 转而又传递给 model_fn。在 custom_estimator.py 中，以下行将创建 Estimator 并设置参数来配置模型。此配置步骤与我们配置 tf.estimator.DNNClassifier（在预创建的 Estimator 中）的方式相似。

In [6]:
def my_model(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    print(predicted_classes.shape)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

## 自定义Estimator

In [7]:
    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.Estimator(
        model_fn=my_model,
        params={
            'feature_columns': my_feature_columns,
            # Two hidden layers of 10 nodes each.
            'hidden_units': [10, 10],
            # The model must choose between 3 classes.
            'n_classes': 3,
        })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\THINK\\AppData\\Local\\Temp\\tmponjwn7__', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000195D277E550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, 1000),
    steps= 50)

INFO:tensorflow:Calling model_fn.
(?,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\THINK\AppData\Local\Temp\tmponjwn7__\model.ckpt.
INFO:tensorflow:loss = 1.8297886, step = 1
INFO:tensorflow:Saving checkpoints for 50 into C:\Users\THINK\AppData\Local\Temp\tmponjwn7__\model.ckpt.
INFO:tensorflow:Loss for final step: 0.33193043.


不懂的地方可以关注 https://www.jianshu.com/p/5495f87107e7 这篇博文